# JsonOutputParser

Hướng dẫn này bao gồm việc triển khai ```JsonOutputParser```.
```JsonOutputParser``` là một công cụ cho phép người dùng chỉ định lược đồ JSON mong muốn. Nó được thiết kế để cho phép LLM (Large Language Model) truy vấn dữ liệu và trả về kết quả ở định dạng JSON tuân theo lược đồ đã chỉ định.
Để đảm bảo rằng LLM xử lý dữ liệu một cách chính xác và hiệu quả, tạo ra JSON ở định dạng mong muốn, mô hình phải có đủ năng lực (ví dụ: trí tuệ). Ví dụ, mô hình llama-70B có năng lực lớn hơn so với mô hình llama-8B, khiến nó phù hợp hơn để xử lý dữ liệu phức tạp.

**[Lưu ý]**

**JSON (JavaScript Object Notation)** là một định dạng trao đổi dữ liệu gọn nhẹ được sử dụng để lưu trữ và cấu trúc dữ liệu. Nó đóng một vai trò quan trọng trong phát triển web và được sử dụng rộng rãi để giao tiếp giữa máy chủ và máy khách. JSON dựa trên văn bản dễ đọc và đơn giản để máy móc phân tích cú pháp và tạo ra.

Dữ liệu JSON bao gồm các cặp key-value. Ở đây, "key" là một chuỗi và "value" có thể là nhiều kiểu dữ liệu khác nhau. JSON có hai cấu trúc chính:

- Object: Một tập hợp các cặp key-value được bao quanh bởi dấu ngoặc nhọn { }. Mỗi key được liên kết với value của nó bằng dấu hai chấm ( : ), và nhiều cặp key-value được phân tách bằng dấu phẩy ( , ).
- Array: Một danh sách các giá trị được sắp xếp theo thứ tự, được bao quanh bởi dấu ngoặc vuông [ ]. Các giá trị trong một mảng được phân tách bằng dấu phẩy ( , ).

```json
{
  "name": "John Doe",
  "age": 30,
  "is_student": false,
  "skills": ["Java", "Python", "JavaScript"],
  "address": {
    "street": "123 Main St",
    "city": "Anytown"
  }
}
```


In [1]:
# Setup environment
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

## Using ```JsonOutputParser``` with ```Pydantic```

If you need to generate output in JSON format, you can easily implement it using LangChain's ```JsonOutputParser```. There are 2 ways to generate output in JSON format: 

- Using ```Pydantic```
- Not using ```Pydantic```

Follow the steps below to implement it:

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [4]:
# Use Pydantic to define the data schema for the output format.
class Topic(BaseModel):
    description: str = Field(description="A concise description of the topic")
    hashtags: str = Field(description="Keywords in hashtag format (at least 2)")

In [5]:
# Write your question
question = "Please explain the severity of global warming."

# Set up the parser and inject the instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "A concise description of the topic", "title": "Description", "type": "string"}, "hashtags": {"description": "Keywords in hashtag format (at least 2)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [6]:
# Set up the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a friendly AI assistant. Answer questions concisely."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# Combine the prompt, model, and JsonOutputParser into a chain
chain = prompt | llm | parser

# Run the chain with your question
answer = chain.invoke({"question": question})

In [7]:
type(answer)

dict

In [8]:
answer

{'description': "Global warming refers to the long-term increase in Earth's average surface temperature due to human activities, such as burning fossil fuels and deforestation. It leads to various severe impacts, including rising sea levels, extreme weather events, loss of biodiversity, and disruptions to ecosystems and agriculture. The severity of global warming is a critical issue that requires urgent action to mitigate its effects and prevent further damage to the planet.",
 'hashtags': 'climatechange, environment, sustainability'}

## Using ```JsonOutputParser``` without ```Pydantic```

You can generate output in JSON format without ```Pydantic```. 

Follow the steps below to implement it:

In [9]:
# Write your question
question = "Please provide information about global warming. Include the explanation in description and the related keywords in `hashtags`."

# Initialize JsonOutputParser
parser = JsonOutputParser()

# Set up the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a friendly AI assistant. Answer questions concisely."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# Inject instruction to prompt
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# Combine the prompt, model, and JsonOutputParser into a chain
chain = prompt | llm | parser

# Run the chain with your question
response = chain.invoke({"question": question})
print(response)

{'description': "Global warming refers to the long-term increase in Earth's average surface temperature due to human activities, such as burning fossil fuels and deforestation, which release greenhouse gases into the atmosphere. This leads to the trapping of heat, causing climate change and various environmental impacts.", 'hashtags': ['climatechange', 'greenhousegases', 'fossilfuels', 'deforestation']}
